In [1]:
aws_credentials_fp = '/home/alexn/Documents/json_credentials/alex_aws_credentials.json'

In [2]:
from minio import Minio
import json
import os

with open(aws_credentials_fp, 'r') as file:
    creds = json.load(file)
creds['region'] = 'us-east-1' # For bucket "test-bucket-sid"


# Retrieve client, assuming you have credentials json file, to access S3
def get_client( ):
    return Minio( 's3.amazonaws.com', secure=True, **creds)

def get_s3_file_tree( bucket_name, root_fp, rel_fp='' ):
    # attributes of "object": ['__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattribute__', \
    #'__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', \
    #'__str__', '__subclasshook__', '__weakref__', 'bucket_name', 'content_type', 'etag', 'is_dir', 'last_modified', 'metadata', \
    #'object_name', 'size']
    
    client = get_client()
    fn_list = []
    
    # 'Objects' contains information on every item in the specified path
    objects = client.list_objects(bucket_name=bucket_name, prefix=root_fp+rel_fp)

    for object in objects:
        # If the 'object' is a directory
        if object.is_dir:
            filepath = object.object_name
            # Get the local filepath (excludes the root fp) to save in fn_list
            filepath_excluding_root = filepath[len(root_fp):]
            fn_list.extend( get_s3_file_tree( bucket_name, root_fp=root_fp, rel_fp=filepath_excluding_root ) )
        # If the 'object' is a file
        else:
            filename = object.object_name
            # Get the filename (exclude the root fp) and save in fn_list
            filename_excluding_root = filename[len(root_fp):]
            fn_list.append( filename_excluding_root )
    
    return fn_list

def get_local_file_tree( local_root_fp, rel_fp='' ):
    file_list_curr_dir = os.listdir( local_root_fp+rel_fp )
    fn_list = []

    for fn in file_list_curr_dir:
        if os.path.isfile(local_root_fp+rel_fp+fn):
            fn_list.append( rel_fp+fn )
        else:
            fn_list.extend( get_local_file_tree( local_root_fp, rel_fp+fn+'/' ) )
            
    return fn_list

In [11]:
s3_file_list = get_s3_file_tree( bucket_name='test-bucket-sid', root_fp='alex_neuroglancer_volumes/' )
print( len(s3_file_list))

local_file_list = get_local_file_tree( '/media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/' )
print( len(local_file_list))

s3_not_local = list( set(s3_file_list)-set(local_file_list) )
local_not_s3 = list( set(local_file_list)-set(s3_file_list) )

156
156


In [12]:
print('Files on S3 but NOT stored locally')
print( s3_not_local )

Files on S3 but NOT stored locally
[]


In [13]:
print('Files stored locally but NOT on S3')
print(local_not_s3 )

Files stored locally but NOT on S3
[]
